In [ ]:
from blueshift.finance import commission, slippage
from blueshift.api import(  symbol,
                          order_target_value,
                          set_commission,
                          set_slippage,
                          schedule_function,
                          date_rules,
                          time_rules,
                          set_stoploss,
                          set_takeprofit
                     )

def initialize(context):
  # universe selection - top 20 most volatile stocks of NIFTY 500
    context.securities = [symbol('OLECTRA'),
                          symbol('ITI'),
                          symbol('TTML'),
                          symbol('HEG'),
                          symbol('IBREALEST'),
                          symbol('CGCL'),
                          symbol('GRAPHITE'),
                          symbol('HFCL'),
                          symbol('FACT'),
                          symbol('SAREGAMA'),
                          symbol('DHANI'),
                          symbol('RTNINDIA'),
                          symbol('RAIN'),
                          symbol('BCG'),
                          symbol('ADANIENT'),
                          symbol('PNB'),
                          symbol('ADANITRANS'),
                          symbol('MASTEK'),
                          symbol('INFIBEAM'),
                          symbol('RADICO')]


  # define strategy parameters
    context.params = {'indicator_lookback':2,
                    'indicator_freq':'1d',
                    'buy_signal':1,
                    'sell_signal':-1,
                    'weight':round(1/len(context.securities),2) # equal distribution of weights among securities
                    }


  # variables to track signals and target portfolio
    context.signals = dict((security,0) for security in context.securities)
    context.target_position = dict((security,0) for security in context.securities)
    context.target_value = dict((security,0) for security in context.securities)




  # set trading cost to 0.3% per dollar and slippage to 0.004%
    set_commission(commission.PerShare(cost=0.003, min_trade_cost=0.0))
    set_slippage(slippage.FixedSlippage(0.00004))
#   print(context.portfolio.portfolio_value)

  # strategy is run everyday at market open
    schedule_function(place_orders, date_rules.every_day(),
                    time_rules.market_open(minutes=0))
   # last trade can be placed at a maximum of 30 minutes before market close
    schedule_function(stop_trading, date_rules.every_day(),
                    time_rules.market_close(minutes=30))
    context.trade = True


def before_trading_start(context, data):
  context.trade = True


def stop_trading(context, data):
  context.trade = False


def place_orders(context, data):
  if not context.trade:
      return

  try:
      price_data_close = data.history(context.securities, 'close',
          context.params['indicator_lookback'],
          context.params['indicator_freq'])




      price_data_open = data.history(context.securities, 'open',
          context.params['indicator_lookback'],
          context.params['indicator_freq'])
  except:
      return




  for security in context.securities:
    #   print(context.portfolio.positions[security].value)
      px_close = price_data_close.loc[:,security].values
      px_open = price_data_open.loc[:,security].values
      context.signals[security] = signal_function(px_close, px_open, context.params)
      if context.signals[security] == context.params['buy_signal']:
        context.target_position[security] = context.params['weight']*context.portfolio.cash
        order_target_value(security, context.target_position[security])
      elif context.signals[security] == context.params['sell_signal']:
            # print(context.portfolio.positions[security].value)
        order_target_value(security,0)
      set_takeprofit(security,"PERCENT",0.5)   # profit booking at 50%
      set_stoploss(security,"PERCENT",0.03)    # stoploss at 3%
      set_stoploss(security,'PERCENT',0.05,trailing = True)





def signal_function(px_close, px_open, params):
   # buy signals

   # gap-up opening
  if px_open[-1] > px_close[-2]:
      return 1
   # 1% price change followed by 1% gain (green candle)
  elif abs((px_close[-2] - px_open[-2])/px_open[-2]) > 0.01 and ((px_close[-1] - px_open[-1])/px_open[-1]) > 0.01:
      return 1


   # sell signals


   # price drop (red candle)
  if px_open[-1] < px_close[-1]:
      return -1
   # 1% gain (green candle) followed by a doji (indicates trend reversal)
  elif abs((px_close[-1] - px_open[-1])/px_open[-1]) < 0.001 and ((px_close[-2] - px_open[-2])/px_open[-2]) > 0.01:
      return -1


  else:
      return 0